This notebook calculates the percentage of category words for a whole dataset

In [ ]:
import nltk
import pickle
import torch
import pandas as pd

from nltk.corpus import stopwords

#lists taken from the attention parser also used to evaluate model performance
position_words =['right', 'left', 'top', 'bottom', 'middle', 'mid', 'front', 'closest', 'nearest', 'center', 'central',
                               'corner', 'upper', 'back', 'far', 'leftmost', 'lower', 'low', 'rightmost',
                               'farthest', 'furthest', 'next', 'last', 'up', 'above', 'below', 'down', 'side']


color_words = ['white', 'green', 'blue', 'red', 'yellow', 'black', 'brown', 'pink', 'dark', 'darker', 'orange',
                            'gray', 'purple', 'beige', 'bright']

size_words = ["big", "bigger", "small", "smaller", "tall", "taller", "large", "larger", "little", "short", "shorter",
                           'tiny', "long", "longer", 'huge']

rel_pos_words=['above', 'about', 'below',"behind" 'beneath', 'beside', 'between', 'by','against', 
               'from', 'through', 'under', 'underneath', 'with','near', 'inside', 'from']

In [2]:

def load_data(model,mode, dataset, split):
#load predicted and gold bounding boxes

    try:

        #the predicted bounding box
        with open(r"/home/users/fschreiber/project/bboxes_"+model+"/"+dataset+"/"+split+"_pred_bbox_list.p","rb") as f:
            pred_bbox_list=list(pickle.load(f))

        if mode=="non_inc":
            #the target bounding box
            with open(r"/home/users/fschreiber/project/bboxes_noninc_"+model+"/"+dataset+"/"+split+"_pred_bbox_list.p","rb") as f:
                target_bbox_list=list(pickle.load(f))

        elif mode == "inc":
            #the target bounding box
            with open(r"/home/users/fschreiber/project/bboxes_"+model+"/"+dataset+"/"+split+"_target_bbox_list.p","rb") as f:
                target_bbox_list=list(pickle.load(f))
        else:
            print("The mode can only be non_inc or inc")
            return -1,-1,-1,-1,-1

        #the number of one sentence split up incrementally ("the left zebra" would have length 3)
        with open(r"/home/users/fschreiber/project/incremental_pickles/length_incremental_units/"+dataset+"_"+split+"_length_unit.p","rb") as f:
            inc_len=pickle.load(f)

        #the original model data split up incrementally
        data_model=torch.load("/home/users/fschreiber/project/ready_inc_data/"+dataset+"/"+dataset+"_"+split+".pth")

        with open(r"/home/users/fschreiber/project/binary_grouped/"+model+"/"+mode+"/"+dataset+split+".p","rb") as f:
            binary_grouped=pickle.load(f)

        
        if mode=="non_inc":
            target_bbox_list=[x for x,y in zip(target_bbox_list,inc_len) for _ in range(y)]
            
        if model=="TVG":
            pred_bbox_list,target_bbox_list=TVG_prep(pred_bbox_list,target_bbox_list)
                    

        return pred_bbox_list,target_bbox_list,inc_len,data_model,binary_grouped
    
    except FileNotFoundError as e:
        #print(e)
        
        return  -1,-1,-1,-1,-1
    


#TVG needs some extra adjustments to fit the same data format as Resc
def TVG_prep(pred_bbox_list,target_bbox_list):
    #print("TVG")
    for ind,(pred,targ) in enumerate (zip (pred_bbox_list,target_bbox_list)):

        pred=pred.view(1,-1)

        pred=xywh2xyxy(pred)
        pred=torch.clamp(pred,0,1)

        pred_bbox_list[ind]=pred

        targ=targ.view(1,-1)
        targ=xywh2xyxy(targ)

        target_bbox_list[ind]=targ
    return pred_bbox_list,target_bbox_list

#copied from TransVG needed to transform the bounding box vectors
def xywh2xyxy(x):
    x_c, y_c, w, h = x.unbind(-1)
    b = [(x_c - 0.5 * w), (y_c - 0.5 * h),
         (x_c + 0.5 * w), (y_c + 0.5 * h)]
    return torch.stack(b, dim=-1)



In [12]:
pred_bbox_list,target_bbox_list,inc_len,model,binary_grouped=load_data("ReSc","inc","unc","testB")


In [13]:
#group sentences that belong to one incremental unit
def group_by_increment(bbox_list,inc_len):
    counter=0
    group_list=[]
    for i in inc_len:
        
        group_list.append(bbox_list[counter:counter+i])
        counter=counter+i
    return group_list

model_group=group_by_increment(model,inc_len)

In [1]:
#splits the sentences by word into wordcategories

def count_category(model_group):
    stop_c,pos_c,rest_c,color_c,size_c,rel_pos_c=(0 for i in range(6))

    for model in model_group:
       
        sent=model[-1][3].split()
        for word in sent:
            
            
            if word in position_words:
                pos_c+=1
               
            elif word in color_words:
                color_c+=1
            elif word in size_words:
                size_c+=1
            elif word in rel_pos_words:
                rel_pos_c+=1
            elif word in stopwords.words():
                stop_c+=1    
           
            else:
                rest_c+=1

    return pos_c,stop_c,size_c,color_c,rel_pos_c,rest_c

#pos_c,stop_c,size_c,color_c,rel_pos_c,rest_c=count_category(model_group)


In [8]:

mode="inc"
model_input="TVG"
split_list=["testB","testA","val","test"]
dataset_list=["unc","unc+","gref_umd","gref"]
#dataset_list=["unc"]

pos_all,color_all,size_all,rel_pos_all,stop_all,rest_all=([] for i in range(6))


#load the data
for file in dataset_list:
    for split in split_list:
    
        #if the data set does not exist ignore
        pred_bbox_list,target_bbox_list,inc_len,model,binary_grouped= load_data(model_input,mode,file,split)
        if pred_bbox_list==-1 or target_bbox_list==-1:
             
             pass
        
        else:
        
            print(file+split)
            model_group=group_by_increment(model,inc_len)

            
            pos_c,stop_c,size_c,color_c,rel_pos_c,rest_c=count_category(model_group) 
            
            all_counter=[pos_c,stop_c,size_c,color_c,rel_pos_c,rest_c]


            pos_all.append(round((pos_c/sum(all_counter)*100),2))
            stop_all.append(round((stop_c/sum(all_counter)*100),2))
            size_all.append(round((size_c/sum(all_counter)*100),2))
            color_all.append(round((color_c/sum(all_counter)*100),2))
            rel_pos_all.append(round((rel_pos_c/sum(all_counter)*100),2))
            rest_all.append(round((rest_c/sum(all_counter)*100),2))
            
            
df_pos_tvg=pd.DataFrame(pos_all)
df_color_tvg=pd.DataFrame(color_all)
df_rel_pos_tvg=pd.DataFrame(rel_pos_all)
df_size_tvg=pd.DataFrame(size_all)
df_stop_tvg=pd.DataFrame(stop_all)
df_rest_tvg=pd.DataFrame(rest_all)  


            

unctestB
unctestA
uncval
unc+testB
unc+testA
unc+val
gref_umdval
gref_umdtest
grefval
